<a href="https://colab.research.google.com/github/israinamdar493-ops/retinalprojectsizz4/blob/main/Retinalai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
from tqdm import tqdm

input_folder = '/content/drive/MyDrive/retinalnewproject/datasets/odir5k'
output_folder = '/content/drive/MyDrive/retinalnewproject/outputs'

os.makedirs(output_folder, exist_ok=True)

for file in tqdm(os.listdir(input_folder)):
    if file.endswith(".jpg") or file.endswith(".png"):
        img = cv2.imread(os.path.join(input_folder, file))
        img_resized = cv2.resize(img, (512,512))
        cv2.imwrite(os.path.join(output_folder, file), img_resized)


100%|██████████| 1/1 [00:00<00:00, 4044.65it/s]


In [ ]:
# Example: read image and normalize
img = cv2.imread('/content/drive/MyDrive/retinalnewproject/datasets/odir5k/datasets/ageDegeneration/image0.png')
img_normalized = img / 255.0  # now pixel values are between 0 and 1


In [ ]:
def augment_image(img):
    import cv2
    import numpy as np

    # 1. Horizontal flip
    img_flip = cv2.flip(img, 1)

    # 2. Rotation (90 degrees)
    img_rot = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

    # 3. CLAHE (Contrast Limited Adaptive Histogram Equalization)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    lab_clahe = cv2.merge((cl,a,b))
    img_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

    return [img_flip, img_rot, img_clahe]

# Example usage
augmented_images = augment_image(img)


In [ ]:
for file in tqdm(os.listdir(input_folder)):
    if file.endswith(".jpg") or file.endswith(".png"):
        img = cv2.imread(os.path.join(input_folder, file))
        aug_imgs = augment_image(img)
        # Save augmented images
        for i, aug in enumerate(aug_imgs):
            filename = file.split('.')[0] + f'_aug{i}.jpg'
            cv2.imwrite(os.path.join(output_folder, filename), aug)


100%|██████████| 1/1 [00:00<00:00, 3057.07it/s]


In [ ]:
import cv2, os
import numpy as np
from tqdm import tqdm

# Your dataset names
DATASETS = ['ODIR5K', 'RFMiD', 'APTOS', 'Messidor2', 'DRIVE', 'CHASEDB1']

# Set paths
base_path = "/content/drive/MyDrive/retinalnewproject/datasets"
output_base = "/content/drive/MyDrive/retinalnewproject/preprocessing"
resize_dim = (512, 512)  # Resize to 512x512

def augment_image(img):
    """Basic data augmentation"""
    aug_list = []
    aug_list.append(cv2.flip(img, 1))                        # Horizontal flip
    aug_list.append(cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)) # Rotate 90
    return aug_list

def preprocess_dataset(dataset):
    input_folder = os.path.join(base_path, dataset)
    output_folder = os.path.join(output_base, dataset)

    if not os.path.exists(input_folder):
        print(f"⚠️ Skipping {dataset}: Input folder not found.")
        return
    os.makedirs(output_folder, exist_ok=True)

    print(f"Processing {dataset}...")

    # ✅ This walks through all subfolders to find images
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)

                if img is None:  # Skip if image can't be read
                    continue

                # Resize and normalize
                img_resized = cv2.resize(img, resize_dim)
                img_normalized = img_resized / 255.0

                # Save resized image
                save_path = os.path.join(output_folder, file)
                cv2.imwrite(save_path, (img_normalized * 255).astype(np.uint8))

                # Data augmentation
                for i, aug in enumerate(augment_image(img_resized)):
                    aug_name = file.split('.')[0] + f'_aug{i}.jpg'
                    cv2.imwrite(os.path.join(output_folder, aug_name), aug)

    print(f"✅ Finished {dataset}\n")

# Run preprocessing for all datasets
for dataset in DATASETS:
    preprocess_dataset(dataset)


Processing ODIR5K...
✅ Finished ODIR5K

Processing RFMiD...
✅ Finished RFMiD

Processing APTOS...
✅ Finished APTOS

Processing Messidor2...
✅ Finished Messidor2

Processing DRIVE...
✅ Finished DRIVE

Processing CHASEDB1...
✅ Finished CHASEDB1



In [ ]:
import os
path = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS"
print("Files inside ODIR5K:", os.listdir(path))


Files inside ODIR5K: ['test.csv', 'train_1.csv', 'valid.csv', 'val_images', 'train_images', 'test_images']


In [ ]:
preprocess_dataset('APTOS/train_images')
preprocess_dataset('APTOS/val_images')
preprocess_dataset('APTOS/test_images')



Processing APTOS/train_images...


100%|██████████| 1/1 [00:00<00:00, 10010.27it/s]



Processing APTOS/val_images...


100%|██████████| 1/1 [00:00<00:00, 15196.75it/s]



Processing APTOS/test_images...


100%|██████████| 1/1 [00:00<00:00, 17697.49it/s]


In [ ]:
import os, shutil

# Base folder where your datasets are stored
BASE_PATH = "/content/drive/MyDrive/retinalnewproject/datasets"

# List of datasets you downloaded
DATASETS = ["ODIR5K","RFMiD","APTOS","Messidor2","DRIVE","CHASEDB1"]

for ds in DATASETS:
    inner_path = os.path.join(BASE_PATH, ds, "datasets")  # The extra nested folder
    outer_path = os.path.join(BASE_PATH, ds)

    if os.path.exists(inner_path):
        print(f"🔄 Moving files from {inner_path} to {outer_path} ...")
        for root, _, files in os.walk(inner_path):
            for f in files:
                src = os.path.join(root, f)
                dst = os.path.join(outer_path, f)
                if not os.path.exists(dst):   # Avoid overwriting
                    shutil.move(src, dst)
        print(f"✅ Moved files for {ds}")
    else:
        print(f"⚠️ No inner 'datasets' folder found for {ds}, skipping.")


🔄 Moving files from /content/drive/MyDrive/retinalnewproject/datasets/ODIR5K/datasets to /content/drive/MyDrive/retinalnewproject/datasets/ODIR5K ...
✅ Moved files for ODIR5K
⚠️ No inner 'datasets' folder found for RFMiD, skipping.
⚠️ No inner 'datasets' folder found for APTOS, skipping.
⚠️ No inner 'datasets' folder found for Messidor2, skipping.
⚠️ No inner 'datasets' folder found for DRIVE, skipping.
⚠️ No inner 'datasets' folder found for CHASEDB1, skipping.


In [1]:
import os
base_path = "/content/drive/MyDrive/retinalnewproject"
print(os.listdir(base_path))


['datasets', 'outputs', 'reports', 'preprocessing', 'segmentation', 'federated', 'ensemble', 'results']


In [2]:
import os

datasets_path = "/content/drive/MyDrive/retinalnewproject/datasets"
print(os.listdir(datasets_path))


['DRIVE', '.ipynb_checkpoints', 'APTOS', 'CHASEDB1', 'Messidor2', 'ODIR5K', 'RFMiD']


In [3]:
train_dir = "/content/drive/MyDrive/retinalnewproject/preprocessing/APTOS"


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),   # can use 512x512 if GPU is strong
    batch_size=16,
    subset='training',
    class_mode='categorical'
)

val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=16,
    subset='validation',
    class_mode='categorical'
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [5]:
!pip install opencv-python tensorflow torch torchvision tqdm matplotlib scikit-learn
